In [1]:
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import FNO
from neuralop.data.datasets import load_darcy_flow_small
from neuralop.utils import count_model_params
from neuralop.training import AdamW
from neuralop.training.incremental import IncrementalFNOTrainer
from neuralop.data.transforms.data_processors import IncrementalDataProcessor
from neuralop import LpLoss, H1Loss

In [2]:
train_loader, test_loaders, output_encoder = load_darcy_flow_small(
    n_train=100,
    batch_size=16,
    test_resolutions=[16, 32],
    n_tests=[100, 50],
    test_batch_sizes=[32, 32],
)

Loading test db for resolution 16 with 100 samples 
Loading test db for resolution 32 with 50 samples 


/Users/sina/Documents/GitHub_Local/XNO/neuraloperator/neuralop/data/datasets/pt_dataset.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(
/Users/sina/D

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
incremental = True
if incremental:
    starting_modes = (2, 2)
else:
    starting_modes = (16, 16)

In [5]:
model = FNO(
    max_n_modes=(16, 16),
    n_modes=starting_modes,
    hidden_channels=32,
    in_channels=1,
    out_channels=1,
)
model = model.to(device)
n_params = count_model_params(model)

In [6]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


# If one wants to use Incremental Resolution, one should use the IncrementalDataProcessor - When passed to the trainer, the trainer will automatically update the resolution
# Incremental_resolution : bool, default is False
#    if True, increase the resolution of the input incrementally
#    uses the incremental_res_gap parameter
#    uses the subsampling_rates parameter - a list of resolutions to use
#    uses the dataset_indices parameter - a list of indices of the dataset to slice to regularize the input resolution
#    uses the dataset_resolution parameter - the resolution of the input
#    uses the epoch_gap parameter - the number of epochs to wait before increasing the resolution
#    uses the verbose parameter - if True, print the resolution and the number of modes
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=16,
    dataset_indices=[2, 3],
    epoch_gap=10,
    verbose=True,
)

data_transform = data_transform.to(device)

Original Incre Res: change index to 0
Original Incre Res: change sub to 2
Original Incre Res: change res to 8


In [7]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()


### N PARAMS ###
 2110305

### OPTIMIZER ###
 AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.008
    lr: 0.008
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###

 * Train: <neuralop.losses.data_losses.H1Loss object at 0x31cf32dd0>

 * Test: {'h1': <neuralop.losses.data_losses.H1Loss object at 0x31cf32dd0>, 'l2': <neuralop.losses.data_losses.LpLoss object at 0x30f69f0d0>}


In [8]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=20,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [9]:
trainer.train(
    train_loader,
    test_loaders,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
)

Training on 100 samples
Testing on [50, 50] samples         on resolutions [16, 32].
Raw outputs of shape torch.Size([16, 1, 8, 8])
[0] time=0.47, avg_loss=0.8300, train_err=11.8578
Eval: 16_h1=0.7720, 16_l2=0.6329, 32_h1=0.8126, 32_l2=0.6275
[1] time=0.40, avg_loss=0.6727, train_err=9.6098
Eval: 16_h1=0.6651, 16_l2=0.5211, 32_h1=0.7453, 32_l2=0.5340
[2] time=0.44, avg_loss=0.5609, train_err=8.0133
Eval: 16_h1=0.7302, 16_l2=0.5144, 32_h1=0.9443, 32_l2=0.5411
[3] time=0.23, avg_loss=0.5267, train_err=7.5238
Eval: 16_h1=0.6545, 16_l2=0.4933, 32_h1=0.8203, 32_l2=0.5120
[4] time=0.24, avg_loss=0.5000, train_err=7.1433
Eval: 16_h1=0.5975, 16_l2=0.4248, 32_h1=0.7602, 32_l2=0.4563
[5] time=0.23, avg_loss=0.4443, train_err=6.3471
Eval: 16_h1=0.5425, 16_l2=0.3776, 32_h1=0.6995, 32_l2=0.4065
[6] time=0.28, avg_loss=0.3906, train_err=5.5802
Eval: 16_h1=0.5395, 16_l2=0.3694, 32_h1=0.7495, 32_l2=0.4146
[7] time=0.22, avg_loss=0.3400, train_err=4.8565
Eval: 16_h1=0.4781, 16_l2=0.3258, 32_h1=0.6661, 

{'train_err': 3.23424232006073,
 'avg_loss': 0.2263969624042511,
 'avg_lasso_loss': None,
 'epoch_train_time': 0.6524578329990618,
 '16_h1': tensor(0.2705),
 '16_l2': tensor(0.2048),
 '32_h1': tensor(0.3730),
 '32_l2': tensor(0.2223)}